<a href="https://colab.research.google.com/github/JotaBlanco/TheValley/blob/main/Advanced_ML_AI/Clase_03_Intro_Redes_Neuronales/03A_Introducci%C3%B3n_Redes_Neuronales_Sin_Resolver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 01 INTRODUCCIÓN
En este notebook introduciremos el concepto de redes neuronales utilizando como ejemplo el ejercicio CHALLENGE sobre churn en una empresa telco.



Notebook por [Javier Blanco Cordero](https://www.linkedin.com/in/javier-blanco-cordero-71373656/).

### Enlaces de interés
*   [Slides de presentación](https://docs.google.com/presentation/d/1vFXqP9QBoipKkGjajyqZEDelGllNueR51kBmXCGWzck/edit?usp=sharing)
*   [Enlace a este notebook](https://colab.research.google.com/github/JotaBlanco/TheValley/blob/main/Advanced_ML_AI/Clase_02_Optimizacion_Modelos/02B_%7C_Ejercicio_CHALLENGE.ipynb)
*   [Enlace a este notebook ejercicio CHALLENGE](https://colab.research.google.com/github/JotaBlanco/TheValley/blob/main/Advanced_ML_AI/Clase_02_Optimizacion_Modelos/02B_%7C_Ejercicio_CHALLENGE.ipynb)

## 0101 Import
Puedes importar aquí las librerías que creas que vas a necesitar.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import graphviz

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

## 0102 Carga el dataset del Churn Telco
Aquí teneis el url del archivo raw del dataset de churn en una empresa Telco. OJO: este es el dataset para entrenar vuestro modelo, sobre el que conocemos el target (si un cliente ha realizado o no churn). 

https://raw.githubusercontent.com/JotaBlanco/TheValley/main/Data/CHURN_DF_train.csv

Importa los datos en un dataframe llamado **dataframe df_churn**.

In [ ]:
# Url archivo raw
url = 'https://raw.githubusercontent.com/JotaBlanco/TheValley/main/Data/CHURN_DF_train.csv'

# Importa csv
df_churn = pd.read_csv(url, sep=',')

# Pequeña limpieza
df_churn.loc[df_churn['TotalCharges']==" ", 'TotalCharges'] = np.NaN
df_churn['TotalCharges'] = df_churn['TotalCharges'].astype(float)

print(len(df_churn))

# Visualización primeras filas
df_churn.head()

# 02 EDA
Para familiarizarte con los datos puedes realizar un pequeño análisis exploratorio visual. Puedes usar los métodos .shape, .info(), .describe(), por ejemplo para entender su tamaño y sus variables. 

Recuerda que puedes encontrar mis clases sobre análisis exploratorio [aquí](https://github.com/JotaBlanco/TheValley/tree/main/EDA/).

In [ ]:
df_churn.info()

In [ ]:
df_churn.describe()

# 03 Preparación de los datos
Normalmente el dataframe contiene nulos, así como variables categóricas y presencia de ciertas variables que probablemente no queramos usar. 

Necesitamos limpiarlo antes de poder entrenar los modelos.

### 0301 Variables Útiles
De entre las variables disponibles, quedémonos con la que queramos utilizar como predictoras para el estudio.

In [ ]:
df_churn.head(2)

In [ ]:
df_churn.columns

In [ ]:
cols = ['gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'tenure', 'PhoneService', 'PaperlessBilling', 'MonthlyCharges',
       'TotalCharges', 'MultipleLines_No', 'MultipleLines_No phone service',
       'MultipleLines_Yes', 'InternetService_DSL',
       'InternetService_Fiber optic', 'InternetService_No', 'StreamingTV_No',
       'StreamingTV_No internet service', 'StreamingTV_Yes',
       'OnlineSecurity_No', 'OnlineSecurity_No internet service',
       'OnlineSecurity_Yes', 'OnlineBackup_No',
       'OnlineBackup_No internet service', 'OnlineBackup_Yes',
       'DeviceProtection_No', 'DeviceProtection_No internet service',
       'DeviceProtection_Yes', 'TechSupport_No',
       'TechSupport_No internet service', 'TechSupport_Yes',
       'StreamingMovies_No', 'StreamingMovies_No internet service',
       'StreamingMovies_Yes', 'Contract_Month-to-month', 'Contract_One year',
       'Contract_Two year', 'PaymentMethod_Bank transfer (automatic)',
       'PaymentMethod_Credit card (automatic)',
       'PaymentMethod_Electronic check', 'PaymentMethod_Mailed check']

cat_cols = ['gender']

In [ ]:
df_churn = df_churn[cols+['Churn']]

### 030102 Dumificación de variables categóricas
Recuerda que debemos transformar las variables categóricas para convertirlas en numéricas.

In [ ]:
df_churn = pd.get_dummies(df_churn[cols+['Churn']], 
                          prefix_sep='_',
                          drop_first=True, 
                          columns=cat_cols)

display(df_churn.shape)

df_churn.head()

### 030103 Limpieza de nulos
Recuerda que puedes mirarlo con dataframe.isna().sum() y si sí hay nulos, puedes rellenarlos con el método .fillna().

In [ ]:
df_churn.isna().sum()

In [ ]:
df_churn['TotalCharges'] = df_churn['TotalCharges'].fillna(df_churn['TotalCharges'].mean())

In [ ]:
df_churn.isna().sum()

### 030104 Estandarización
Normalizamos (estandarizamos) las variables numéricas. Scklearn propone distintos métodos:

*   [sklearn.preprocessing.StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html)
*   [sklearn.preprocessing.MinMaxScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html)
*   [sklearn.preprocessing.normalize](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.normalize.html)

In [ ]:
df_churn.head()

In [ ]:
# columnas no binarias 
display(df_churn.loc[:,df_churn.isin([0,1]).all()==False].columns)

# non-binary cols
non_binary_cols = ['tenure', 'MonthlyCharges', 'TotalCharges']

In [ ]:
from sklearn.preprocessing import StandardScaler

# Inicializamos objeto de normalización
scaler = StandardScaler()

# "Entrenamos" el objeto 
scaler.fit(df_churn[non_binary_cols])

# Transformamos las columnas no binarias
df_churn[non_binary_cols] = scaler.transform(df_churn[non_binary_cols])

df_churn.head()

### 030105 Train - test

In [ ]:
# Primero generamos la matriz X con las variables predictoras 
# y el vector y con la variable a predecir
X = df_churn.drop('Churn',axis=1)
y = df_churn['Churn']

# Comprueba sus tamaños
X.shape, y.shape

In [ ]:
# Haz la partición train-test
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size=0.33,
                                                    random_state=42)

# Comprueba sus tamaños
X_train.shape, X_test.shape, y_train.shape, y_test.shape

# 04 Entrenamos una red neuronal con Keras

Aquí podéis encontrar [un buen notebook ejemplo](https://colab.research.google.com/github/keras-team/keras-io/blob/master/guides/ipynb/sequential_model.ipynb) (de la documentación de Keras) sobre cómo definir redes neuronales secuenciales. 

## 0401 Importamos Keras

In [ ]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense

## 0402 Definimos arquitectura
Más info sobre la función de activación softmax [aquí](https://en.wikipedia.org/wiki/Softmax_function).

In [ ]:
# Inicializamos la red neuronal
modelo = Sequential([
    Dense(1000, activation='relu', input_shape=(X.shape[1],)),
    Dense(1000, activation='relu'),
    Dense(1000, activation='relu'),
    Dense(1, activation="sigmoid")
    ], name = 'Mi_primera_red_neuronal')

modelo

In [ ]:
modelo.summary()

In [ ]:
keras.utils.plot_model(modelo, "imagen.png", show_shapes=True)

## 0403 Compilamos

In [ ]:
modelo.compile(optimizer = 'sgd', 
               loss = 'binary_crossentropy', 
               metrics = ['accuracy'])

modelo

## 0404 Entrenamos
[Colab notebook](https://colab.research.google.com/github/keras-team/keras-io/blob/master/guides/ipynb/training_with_built_in_methods.ipynb#scrollTo=IFz5UrHJC4SF) muy completo de la documentacion de Keras sobre el entrenamiento de redes neuronales.

In [ ]:
modelo.fit(X_train, y_train, epochs=25, batch_size=50)
modelo

In [ ]:
# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = modelo.evaluate(X_test, y_test, batch_size=128)
print("test loss, test acc:", results)

## 0405 Predecimos


In [ ]:
# Usamos la red neuronal para predecir sobre el dataset de entrenamiento
y_pred_train = np.round(modelo.predict(X_train), 0)

# Usamos la red neuronal para predecir sobre el dataset de test
y_pred_test = np.round(modelo.predict(X_test), 0)

In [ ]:
# Calidad predicción train
ac_train = round(accuracy_score(y_train, y_pred_train), 4)
print('Precisión en set de entrenamiento :', ac_train)

# Calidad predicción test
ac_test = round(accuracy_score(y_test, y_pred_test), 4)
print('Precisión en set de test :', ac_test)

# Degradación de la predicción
print('Degradación: ' + str(round((ac_test-ac_train)/ac_train*100,2)) + '%')

# 05 EJERCICIO
Entrena una red neuronal que prediga si se da o no una complicación en una operación quirúrgica.

## 0501 Carga el dataset de complicaciones quirúrgicas
Aquí teneis el url del archivo raw del dataset: 

https://raw.githubusercontent.com/JotaBlanco/TheValley/main/Data/complicacion_quirurgica.csv

Importa los datos en un dataframe llamado **dataframe df**.

In [ ]:
# Url archivo raw
url = 'https://raw.githubusercontent.com/JotaBlanco/TheValley/main/Data/complicacion_quirurgica.csv'

# Importa csv
df = pd.read_csv(url, sep=',')

# Visualización primeras filas
df.head()

In [ ]:
df.info()

## 0502 Estandarización
Normaliza las variables numéricas para que estén en la misma escala.


In [ ]:
# columnas no binarias 
display(df.loc[:,df.isin([0,1]).all()==False].columns)

# non-binary cols
non_binary_cols = ['bmi', 'Age', 'asa_status', 'baseline_charlson', 'ahrq_ccs',
       'ccsComplicationRate', 'ccsMort30Rate', 'complication_rsi', 'dow',
       'hour', 'month', 'moonphase', 'mortality_rsi', 'race']

In [ ]:
# normaliza las columnas numéricas no binarias


## 0503 Train - test
Separa el dataset en train y test.

## 0504 Entrena una red neuronal
Prueba de hecho varias, con distintas arquitecturas.